In [2]:
# Import modules

# Sci computing
import numpy as np
import scipy as sp
import seawater as sw
import scipy.sparse.linalg as sla

# Parallel comupting
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar

# For Data
import netCDF4 as nc
import xarray as xr

# Plotting stuff
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as grdspc
import cmocean as cm

# Gen stuff
from datetime import date
today = date.today()

In [ ]:
# Load data from HYCOM

# Get lat lon
url_latlon = 'https://tds.hycom.org/thredds/dodsC/GOMl0.04/expt_02.2/mlayer/1/ssh?Latitude[0:1:384],Longitude[0:1:540]'
ds_latlon = xr.open_dataset(url_latlon)
#ds_latlon.to_netcdf("hycom_latlon.nc") 

# Get depth
url_depth = 'https://tds.hycom.org/thredds/dodsC/GOMl0.04/expt_02.2/z3d/1/t?Depth[0:1:32]'
ds_depth = xr.open_dataset(url_depth)
#ds_depth.to_netcdf("hycom_depth.nc")

In [4]:
ds_lon = ds_latlon['Longitude'][:]
ds_lat = ds_latlon['Latitude'][:]
ds_z = ds_depth['Depth'][:] 

# finding the index in lon array that corresponds to 90W
ind90 = list(np.where(ds_lon >= -90)) 
ind83 = list(np.where(ds_lon >= -83))
nlon90 = ind90[0][0]
nlon83 = ind83[0][0] + 1
lon = ds_lon[nlon90:nlon83]

# finding the indices in lat array that correspond to 22W and 28W to only grab data from this region
ind22 = list(np.where(ds_lat >= 22))
nlat22 = ind22[0][0] 
ind28 = list(np.where(ds_lat >= 28))
nlat28 = ind28[0][0] + 1
lat = ds_lat[nlat22:nlat28]

ind2k = list(np.where(ds_z >= 2000))
n2k = ind2k[0][0] + 1
depth = ds_z[:n2k]

In [ ]:
# Get SSH data for full 18 year cycle (first cycle)

url_ssh = f'https://tds.hycom.org/thredds/dodsC/GOMl0.04/expt_02.2/mlayer/1?ssh[0:1:6573][{nlat22}:1:{nlat28-1}][{nlon90}:1:{nlon83-1}]'
ds_ssh  = xr.open_dataset(url_ssh, chunks={'MT': 30})
#ds_ssh.to_netcdf("./hycom_data/hycom_ssh.nc")

In [ ]:

url_temp = f'https://tds.hycom.org/thredds/dodsC/GOMl0.04/expt_02.2/z3d/1/t?temperature[0:1:6573][0:1:{n2k-1}][{nlat22}:1:{nlat28-1}][{nlon90}:1:{nlon83-1}]'
ds_temp = xr.open_dataset(url_temp, chunks={'MT': 30})
#ds_temp.to_netcdf("./hycom_data/hycom_temp.nc")

In [ ]:
url_sal = f'https://tds.hycom.org/thredds/dodsC/GOMl0.04/expt_02.2/z3d/1/s?salinity[0:1:6573][0:1:{n2k-1}][{nlat22}:1:{nlat28-1}][{nlon90}:1:{nlon83-1}]'
ds_sal = xr.open_dataset(url_sal, chunks={'MT': 30})
#ds_sal.to_netcdf("./hycom_data/hycom_sal.nc")